In [2]:
import pandas as pd
import numpy as np

from talkingdata import run
from models import PhoneBrandEstimator

%load_ext autoreload
%autoreload 2

In [36]:
gatrain = pd.read_csv('data/gender_age_train.csv')
pbrand = pd.read_csv('data/phone_brand_device_model.csv')

In [37]:
gatest = pd.read_csv('data/gender_age_test.csv')

In [38]:
pbrand.phone_brand.describe()

count     187245
unique       131
top           小米
freq       43210
Name: phone_brand, dtype: object

In [39]:
gaptrain = gatrain.merge(pbrand.drop_duplicates('device_id',keep='first'), on='device_id')

In [40]:
gaptrain.columns

Index(['device_id', 'gender', 'age', 'group', 'phone_brand', 'device_model'], dtype='object')

In [41]:
translations = pd.read_csv('data/phone_brand_chinese_to_english.csv', sep=',')

In [42]:
translations.head()

,ch,eng
0,三星,samsung
1,天语,Ktouch
2,海信,hisense
3,联想,lenovo
4,欧比,obi


In [43]:
translations.ch.describe()

count     55
unique    55
top       海尔
freq       1
Name: ch, dtype: object

Como podemos ver, hay muchos menos modelos con traducciones (55) que el total (131)

In [44]:
for chb, engb in zip(translations['ch'], translations['eng']):
    gaptrain.replace(to_replace=chb, value=engb, inplace=True)

#gaptrain.replace(to_replace=translations['ch'], value=translations['eng'], inplace=True)

Acabo de descubrir que 小米 es la traduccion de Xiaomi. Y 红米 es Red Rice

In [45]:
gaptrain.replace(to_replace=['小米', '红米'], value=['Xiaomi', 'Red Rice'], inplace=True)

In [46]:
gaptrain.device_id = gaptrain.device_id.astype('category')
gaptrain.device_model = gaptrain.device_model.astype('category')
gaptrain.phone_brand = gaptrain.phone_brand.astype('category')

In [47]:
results = run(PhoneBrandEstimator, gaptrain.dropna(), dump_path='data/')

In [48]:
results.to_csv('data/results_pbestimator.csv', index=False)

Ahora vamos a predecir sobre el test set.

In [49]:
gaptest = gatest.merge(pbrand.drop_duplicates('device_id',keep='first'), on='device_id')

for chb, engb in zip(translations['ch'], translations['eng']):
    gaptest.replace(to_replace=chb, value=engb, inplace=True)
gaptest.replace(to_replace=['小米', '红米'], value=['Xiaomi', 'Red Rice'], inplace=True)

In [50]:
results, rpred = run(PhoneBrandEstimator, gaptrain.dropna(), dump_path='data/', valid=gaptest)

In [51]:
rpred.shape

(448284, 12)

In [52]:
len(gatest) * 4

448284

In [56]:
pred = pd.DataFrame(rpred[:len(rpred)/4], columns=results.columns[2:])
pred.insert(0, 'device_id', gatest.device_id)

TypeError: cannot do slice indexing on <class 'pandas.indexes.range.RangeIndex'> with these indexers [28017.75] of <class 'float'>

In [57]:
pred.to_csv('data/valid_predictions.csv', index=False)